# Basic SCS Interface

This tutorial only covers the specifics of the Python SCS interface. For background and descriptions of the optimization problem being solved, the matrix `A`, vectors `b` and `c`, and the `cone` dictionary, please see the [SCS README](https://github.com/cvxgrp/scs).

In [1]:
import scs

We'll load data for an example problem.

In [2]:
m = 2000
data, cone = scs.examples.l1(m=m)
print 'data: ', data
print 'cone: ', cone

Data:  {'A': <10000x8000 sparse matrix of type '<type 'numpy.float64'>'
	with 816000 stored elements in Compressed Sparse Column format>, 'c': array([ 0.,  0.,  0., ...,  1.,  1.,  1.]), 'b': array([-1.06370562,  1.2659559 ,  0.0602342 , ...,  0.        ,
        0.        ,  0.        ])}
Cone:  {'l': 8000, 'f': 2000}


Use `scs.solve()` to compute the solution. `data` and `cone` are required arguments. Solver settings can be passed as keyword arguments. `result` is a dictionary with keys `'x'`, `'y'`, `'s'` (corresponding to problem variables), and `'info'` (a `dict` giving solver status information).

In [3]:
%%time
result = scs.solve(data, cone, use_indirect=True, verbose=False)
print('---###---')

---###---
CPU times: user 16.6 s, sys: 177 ms, total: 16.7 s
Wall time: 17.5 s


In [4]:
print result.keys()
result['info']

['y', 'x', 's', 'info']


{'dobj': 236.77453426584182,
 'iter': 540,
 'pobj': 236.77533277641038,
 'relGap': 1.6826694600814932e-06,
 'resDual': 0.000992509383044581,
 'resInfeas': 11.836075870338009,
 'resPri': 0.0006946113096443507,
 'resUnbdd': nan,
 'setupTime': 70.512216,
 'solveTime': 17383.026628,
 'status': u'Solved',
 'statusVal': 1}

A **copy** of the SCS default solver settings can be seen by calling `scs.default_settings()`.

In [5]:
scs.default_settings()

{'alpha': 1.5,
 'cg_rate': 2.0,
 'eps': 0.001,
 'max_iters': 2500,
 'normalize': True,
 'rho_x': 0.001,
 'scale': 1.0,
 'use_indirect': False,
 'verbose': True,
 'warm_start': False}

# Warm-Starting

The solver can be warm-started with vectors `x`, `y`, and `s` which are expected to be close to the solution. This can reduce the number of iterations needed to converge.

Pass a dictionary with keys `'x'`, `'y'`, and `'s'` to the `sol` parameter of `scs.solve()`, and set the option `warm_start=True`. Note that all three vectors are needed to warm-start.

Below, we use the previous solution to warm-start the solver on the same problem, and see that this allows the solver to exit after 0 iterations.

In [6]:
%%time
sol = dict(x=result['x'],y=result['y'],s=result['s'])
result = scs.solve(data, cone, sol=sol, verbose=True, warm_start=True, use_indirect=False)
print('---###---')

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 816000
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 8000, constraints m = 10000
Cones:	primal zero / dual free vars: 2000
	linear vars: 8000
Setup time: 3.94e+00s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.91e-04  9.92e-04  1.44e-06  2.37e+02  2.37e+02  0.00e+00  3.52e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 3.54e-02s
	Lin-sys: nnz in L factor: 2837000, avg solve 

We can confirm the number of iterations by inspecting the `'info'` dictionary. Note that even though
the solver needed 0 iterations to converge, it still had to perform a matrix factorization (since `use_indirect=False`), which took about 4 seconds.

In [7]:
result['info']

{'dobj': 236.7732123928627,
 'iter': 0,
 'pobj': 236.77389637859108,
 'relGap': 1.4413442116573387e-06,
 'resDual': 0.000991580023177425,
 'resInfeas': 11.8361571901985,
 'resPri': 0.0006905369290310406,
 'resUnbdd': nan,
 'setupTime': 3937.024572,
 'solveTime': 35.38458,
 'status': u'Solved',
 'statusVal': 1}

# Factorization Caching Interface

When using the direct method (`use_indirect=False`), we can cache the matrix factorization of `A`, and reuse it across several solves. This is useful when solving a sequence or family of problems where `A` is fixed, but `b` and `c` may change.

The `scs.Workspace` object caches the matrix factorization for us, and allows us to call the solver many times with different values for `b` and `c`. We can also optionally warm-start the solver, and change **some** of the solver settings between solves.

Below, we initialize the `Workspace` object with the same data as above, and note that the setup time (factorization time) is still approximately 4 seconds. Note that the `Workspace` defaults to the direct (factorization) method because `use_indirect=False`, unless the user specifies otherwise.

In [8]:
%%time
work = scs.Workspace(data, cone)
print('---###---')

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 816000
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 8000, constraints m = 10000
Cones:	primal zero / dual free vars: 2000
	linear vars: 8000
Setup time: 4.31e+00s
---###---
CPU times: user 4.15 s, sys: 65.1 ms, total: 4.22 s
Wall time: 4.31 s


## `work.info`

`work.info` will give a **copy** of the `info` dictionary, showing solver status information. Since only the setup has run, only the `setupTime` key is nonzero.

In [9]:
work.info

{'dobj': 0.0,
 'iter': 0,
 'pobj': 0.0,
 'relGap': 0.0,
 'resDual': 0.0,
 'resInfeas': 0.0,
 'resPri': 0.0,
 'resUnbdd': 0.0,
 'setupTime': 4309.260194,
 'solveTime': 0.0,
 'status': u'',
 'statusVal': 0}

## `work.settings`

The `Workspace` object records changes to the user-specified settings. They can be inspected and modified through the
`work.settings` dictionary. When the solver is run, it will operate based on the current `settings`.

Some settings cannot be changed once the `Workspace` object is initialized:
- `use_indirect`
- `normalize`
- `scale`
- `rho_x`

In [10]:
work.settings

{'alpha': 1.5,
 'cg_rate': 2.0,
 'eps': 0.001,
 'max_iters': 2500,
 'normalize': True,
 'rho_x': 0.001,
 'scale': 1.0,
 'use_indirect': False,
 'verbose': True,
 'warm_start': False}

## `work.data`

The vectors `b` and `c` can be modified through the `work.data` dictionary. Please do not modify the matrix `A` after the `Workspace` has been initialized. Changing it will give incorrect results, since the cached matrix factorization depends on the original `A`.

In [11]:
work.data

{'A': <10000x8000 sparse matrix of type '<type 'numpy.float64'>'
 	with 816000 stored elements in Compressed Sparse Column format>,
 'b': array([-1.06370562,  1.2659559 ,  0.0602342 , ...,  0.        ,
         0.        ,  0.        ]),
 'c': array([ 0.,  0.,  0., ...,  1.,  1.,  1.])}

## `work.sol`
The solution vectors `x`, `y`, and `s` are stored in `work.sol`. These are the `numpy` arrays where the solution will be written. The user is free to modify this dictionary and/or the vectors themselves, but do ensure the sizes of the vectors remain unchanged.

The user may change these vectors at any time to write the solution to a different location, or to input warm-starting vectors.

By default, the `Workspace()` constructor will create empty `numpy` arrays to populate `work.sol`. The user can supply their own by passing a dictionary to the `sol` keyword argument of `Workspace()`.

In [12]:
work.sol

{'s': array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 'x': array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 'y': array([ 0.,  0.,  0., ...,  0.,  0.,  0.])}

## `work.solve()`

`work.solve()` will call the solver based on the current `settings`, `sol` (if `warm_start=True`), `data['b']` and `data['c']`.

Calling the solver below, note that we skip the setup step, and go right to the iterative procedure.

In [13]:
%%time
result = work.solve()
print('---###---')

----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.92e-02 
   100| 7.99e-03  1.05e-02  3.04e-05  2.37e+02  2.37e+02  1.13e-14  1.31e+00 
   200| 3.13e-03  4.02e-03  7.04e-06  2.37e+02  2.37e+02  1.13e-14  3.22e+00 
   300| 1.77e-03  2.30e-03  1.38e-06  2.37e+02  2.37e+02  1.13e-14  4.33e+00 
   400| 1.11e-03  1.53e-03  2.07e-06  2.37e+02  2.37e+02  1.13e-14  5.36e+00 
   500| 7.85e-04  1.09e-03  5.64e-07  2.37e+02  2.37e+02  1.13e-14  6.45e+00 
   540| 6.89e-04  9.95e-04  1.08e-06  2.37e+02  2.37e+02  1.13e-14  6.88e+00 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 6.88e+00s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 1.23e-02s
	Cones: avg projection time: 3.58e-05s


Note that the solution is stored in the `work` object in `work.sol`.

In [14]:
work.sol

{'s': array([  5.71284582e-17,  -1.17107198e-16,   1.10606304e-16, ...,
         -7.06831992e-18,   5.30123994e-18,   2.41232642e-01]),
 'x': array([  7.10379611e-05,   2.99788766e-05,  -1.08941016e-05, ...,
         -4.98375019e-08,  -4.98375019e-08,   1.20615895e-01]),
 'y': array([ 0.19930045, -0.21738546,  0.24355537, ...,  0.89722381,
         0.7904324 ,  0.        ])}

## Caching and warm-starting

We can solve the same problem with the cached matrix factorization and a warm-started solution by calling
`work.solve(warm_start=True)`. This should require 0 setup-time (since its already been done) and 0 iterations (since we are warm-starting already at the solution stored in `work.sol`).

In [15]:
%%time
result = work.solve(warm_start=True)
print('---###---')

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.86e-04  9.92e-04  1.03e-06  2.37e+02  2.37e+02  0.00e+00  3.49e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 3.51e-02s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 2.98e-02s
	Cones: avg projection time: 8.68e-05s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 5.5713e-16, dist(y, K*) = 0.0000e+00, s'y/m = 5.2014e-20
|Ax + s - b|_2 / (1 + |b|_2) = 6.8572e-04
|A'y + c|_2 / (1 + |c|_2) = 9.9205e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 1.0261e-06
----------------------------------------------------------------------------
c'x = 236.7747, -b'y = 236.7742
---###---
CPU times: user 37.3 ms, s

## Perturbed warm-starting

For a more interesting example of warm-starting, we perturb the `b` vector a bit and try to re-solve.
However, if we warm-start from the previous solution to the unperturbed problem, we can expect to only need a few iterations to "correct" for the perturbation and obtain the new solution.

In [16]:
%%time
# make a small perturbation to b
work.data['b'][:m] += .01
result = work.solve(warm_start=True)
print('---###---')

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 7.63e-04  1.22e-03  3.53e-06  2.37e+02  2.37e+02  0.00e+00  3.55e-02 
    20| 6.89e-04  9.90e-04  3.63e-06  2.37e+02  2.37e+02  3.02e-14  2.43e-01 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.43e-01s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 1.08e-02s
	Cones: avg projection time: 3.59e-05s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 5.4599e-16, dist(y, K*) = 0.0000e+00, s'y/m = -8.3117e-20
|Ax + s - b|_2 / (1 + |b|_2) = 6.8888e-04
|A'y + c|_2 / (1 + |c|_2) = 9.8963e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 3.6280e-06
------------------------------------------------------------------

If we attempt to solve the problem without warm-starting, it will require many more iterations (and a longer solve time).

In [17]:
%%time
result = work.solve(warm_start=False)
print('---###---')

----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.44e-02 
   100| 7.98e-03  1.05e-02  2.65e-05  2.37e+02  2.37e+02  2.26e-14  1.13e+00 
   200| 3.13e-03  4.01e-03  9.53e-06  2.37e+02  2.37e+02  2.26e-14  2.13e+00 
   300| 1.77e-03  2.30e-03  1.75e-06  2.37e+02  2.37e+02  2.26e-14  3.30e+00 
   400| 1.11e-03  1.53e-03  1.21e-06  2.37e+02  2.37e+02  2.26e-14  4.39e+00 
   500| 7.54e-04  1.12e-03  1.79e-07  2.37e+02  2.37e+02  2.26e-14  5.39e+00 
   540| 6.97e-04  9.78e-04  1.54e-06  2.37e+02  2.37e+02  2.26e-14  5.80e+00 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 5.80e+00s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 1.03e-02s
	Cones: avg projection time: 3.35e-05s


If we revert even futher and try to solve **without** factorization caching or warm-starting, we can expect an even longer solve time.

In [18]:
%%time
data = work.data
result = scs.solve(data, cone, warm_start=False)
print('---###---')

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 816000
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 8000, constraints m = 10000
Cones:	primal zero / dual free vars: 2000
	linear vars: 8000
Setup time: 4.11e+00s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.38e-02 
   100| 7.98e-03  1.05e-02  2.65e-05  2.37e+02  2.37e+02  2.26e-14  1.22e+00 
   200| 3.13e-03  4.01e-03  9.53e-06  2.37e+02  2.37e+02  2.26e-14  2.43e+00 
   300| 1.77e-03  2.30e-03  1.75e-06  2.37e+0